##**Тема**: Машинный перевод текста. Сравнение моделей и их оценка.

**Цель**: Выполнить машинный перевод текста, сравнить производительность нескольких моделей машинного перевода с английского на русский.

**Целевая аудитория**: исследователи в области NLP, разработчики машинного перевода, ученые, переводческие агентства, международные организации и компании, которым необходим оперативный перевод материалов, документации, референсов, а также перевод текстов для локализации сайтов.

**Задачи**:
1.	Подготовка данных и их обработка
2.	Элайнмент(выравнивание) датасета для обучения
3.	Настройка модели и параметров обучения
4.	Обучение модели
5.	Инференс и оценка качества перевода
6.	Расчет метрик SacreBLEU (в коде по ошибке назван chrf), COMET и METEOR
7.	Сравнение результатов до и после дообучения


**Сбор данных**

**Источники данных**: публикация МАГАТЭ в свободном доступе на официальном сайте МАГАТЭ https://www.iaea.org/ru/publications/14836/obrashchenie-s-izyatymi-iz-upotrebleniya-zakrytymi-radioaktivnymi-istochnikami

**Корпус текстов:** публикация МАГАТЭ "Обращение с изъятыми из употребления закрытыми радиоактивными источниками“. Исходные данные представляют собой пары предложений на русском и английском языках.

**Предобработка**: выполнение выравнивания (alignment) английской и русской версий текстов в программе ABBYY Aligner.



In [ ]:
USER = 'elsharafutdinova'

import os
os.environ['CUDA_DEVICE_ORDER']='PCI_BUS_ID'
os.environ['CUDA_VISIBLE_DEVICES']='6, 5' #'0,1'
os.environ['HF_HOME'] = f'/data/{USER}/.cache'
os.environ['HF_HUB_CACHE'] = f'/data/{USER}/.cache'

import sys
import pandas as pd
import torch
from transformers import AutoTokenizer
from transformers import AutoModelForCausalLM, TrainingArguments, BitsAndBytesConfig
from trl import SFTTrainer, DataCollatorForCompletionOnlyLM
from peft import LoraConfig, get_peft_model
from peft import PeftModel
from transformers import pipeline
from torch.utils.data import DataLoader
from tqdm import tqdm
from copy import copy


sys.path.append(f'/data/{USER}/workspace')

In [ ]:
import pandas as pd

# Данные: предложения на русском и их переводы на английский
data = {
    'Russian': [
        'Это тестовое предложение.',
        'Как твои дела?',
        'Погода сегодня отличная.',
        'Мне нравится программировать.',
        'Добро пожаловать в наш офис.'
    ],
    'English': [
        'This is a test sentence.',
        'How are you?',
        'The weather is great today.',
        'I like programming.',
        'Welcome to our office.'
    ]
}

# Создание датафрейма
df = pd.DataFrame(data)

# Печать датафрейма
print(df)

                         Russian                      English
0      Это тестовое предложение.     This is a test sentence.
1                 Как твои дела?                 How are you?
2       Погода сегодня отличная.  The weather is great today.
3  Мне нравится программировать.          I like programming.
4   Добро пожаловать в наш офис.       Welcome to our office.


In [ ]:
from datasets import Dataset, DatasetDict

dataset = Dataset.from_pandas(df)

# Для демонстрации создадим DatasetDict с обучающим и тестовым наборами данных
# Разделим данные на 80% для обучения и 20% для тестирования
train_test_split = dataset.train_test_split(test_size=0.2)

# Создаем DatasetDict
dataset_dict = DatasetDict({
    'train': train_test_split['train'],
    'test': train_test_split['test']
})

# Выводим DatasetDict для проверки
print(dataset_dict)

DatasetDict({
    train: Dataset({
        features: ['Russian', 'English'],
        num_rows: 4
    })
    test: Dataset({
        features: ['Russian', 'English'],
        num_rows: 1
    })
})


In [ ]:
df = pd.read_csv('elena.csv')

In [ ]:
df.iloc[16]['Russian']

'The copyright has since been extended by the World Intellectual Property Organization (Geneva) to include electronic and virtual intellectual property. |Впоследствии авторские права были распространены Всемирной организацией интеллектуальной собственности (Женева) также на интеллектуальную собственность в электронной и виртуальной форме. |'

In [ ]:
rus = []
eng = []
for i in range(10, df.shape[0]):
  try:
    cur_rus = df.iloc[i]['Russian'].split('|')[1]
    cur_eng = df.iloc[i]['Russian'].split('|')[0]
    #print(cur_eng, cur_rus)
    rus.append(cur_rus)
    eng.append(cur_eng)
  except:
    continue

In [ ]:

# Создаем DataFrame из массивов
df_rus = pd.DataFrame({
    'Russian': rus,
    'English': eng
})

In [ ]:
rus = []
eng = []
for i in range(df.shape[0]):
  try:
    cur_rus = df.iloc[i]['English'].split('|')[1]
    cur_eng = df.iloc[i]['English'].split('|')[0]
    #print(cur_eng, cur_rus)
    rus.append(cur_rus)
    eng.append(cur_eng)
  except:
    continue

In [ ]:
df_eng = pd.DataFrame({
    'Russian': rus,
    'English': eng
})

In [ ]:
# Объединяем DataFrame по строкам (по умолчанию)
df_final = pd.concat([df_rus, df_eng])

In [ ]:
# df_final['Russian'] = df_final['Russian'].str.lower()
# df_final['English'] = df_final['English'].str.lower()

In [ ]:
df_final = df_final.reset_index(drop=True)

from datasets import Dataset, DatasetDict

dataset = Dataset.from_pandas(df_final)

# Для демонстрации создадим DatasetDict с обучающим и тестовым наборами данных
# Разделим данные на 80% для обучения и 20% для тестирования
train_test_split = dataset.train_test_split(test_size=0.2)

# Создаем DatasetDict
dataset_dict = DatasetDict({
    'train': train_test_split['train'],
    'test': train_test_split['test']
})

# Выводим DatasetDict для проверки
print(dataset_dict)

DatasetDict({
    train: Dataset({
        features: ['Russian', 'English'],
        num_rows: 3179
    })
    test: Dataset({
        features: ['Russian', 'English'],
        num_rows: 795
    })
})


In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test = train_test_split(df_final, test_size=0.25, random_state=42)

In [ ]:
X_train.shape, X_test.shape

((2980, 2), (994, 2))

In [ ]:
X_train = X_train.reset_index()

In [ ]:
X_test = X_test.reset_index()

In [ ]:
import datasets
from datasets import arrow_dataset

test_books = arrow_dataset.Dataset.from_dict({"id": list(X_test.index), "en": list(X_test['English']), "ru": list(X_test['Russian'])})
train_books = arrow_dataset.Dataset.from_dict({"id": list(X_train.index), "en": list(X_train['English']), "ru": list(X_train['Russian'])})

books = datasets.DatasetDict({"train":train_books,"test":test_books})

In [ ]:
books['train'].to_csv('elena_data_v1/train.csv', index=False)
books['test'].to_csv('elena_data_v1/train.csv', index=False)

Creating CSV from Arrow format:   0%|          | 0/3 [00:00<?, ?ba/s]

998235

In [ ]:
from transformers import AutoTokenizer
from transformers import AutoTokenizer
from transformers import AutoModelForCausalLM, TrainingArguments, BitsAndBytesConfig
from trl import SFTTrainer, DataCollatorForCompletionOnlyLM

checkpoint = "haoranxu/ALMA-7B-R"
model = AutoModelForCausalLM.from_pretrained("haoranxu/ALMA-7B-R", device_map="auto")
tokenizer = AutoTokenizer.from_pretrained("haoranxu/ALMA-7B-R", padding_side='left')

In [ ]:
def formatting_prompts_func(example):
    output_texts = []
    for i in range(len(example)):
        text = f"<s>[INST] Translate this from English to Russian:\nEnglish: {example['en'][i]} \nRussian: [/INST] {example['ru'][i]} </s>"
        output_texts.append(text)
    return output_texts

In [ ]:
from peft import LoraConfig, get_peft_model

In [ ]:
response_template = "[/INST]"
collator = DataCollatorForCompletionOnlyLM(response_template, tokenizer=tokenizer)
peft_config = LoraConfig(
    lora_alpha=16,
    lora_dropout=0.1,
    r=32,
    bias="none",
    target_modules=["q_proj","k_proj","v_proj","o_proj"],
    task_type="CAUSAL_LM",
)

model = get_peft_model(model, peft_config)
model.print_trainable_parameters()
training_args = TrainingArguments(
    output_dir="elena_data_v1",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    weight_decay=0.01,
    save_total_limit=10,
    num_train_epochs=4,
    fp16=True,
    eval_accumulation_steps=5, # use if OOM & trainer runs>=10 mins and crushes. Less steps -> slower training
    logging_steps=300,
    report_to='tensorboard',
)

trainer = SFTTrainer(
    model=model,
    args=training_args,
    peft_config=peft_config,
    train_dataset=books['train'],
    eval_dataset=books['test'],
    tokenizer=tokenizer,
    formatting_func=formatting_prompts_func,
    data_collator=collator,
)

In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss
1,No log,0.570848
2,No log,0.565314
3,No log,0.562394
4,No log,0.561348


TrainOutput(global_step=12, training_loss=0.571537971496582, metrics={'train_runtime': 12.4035, 'train_samples_per_second': 2.902, 'train_steps_per_second': 0.967, 'total_flos': 233526525640704.0, 'train_loss': 0.571537971496582, 'epoch': 4.0})

## Считывание модели

In [ ]:
def get_model_tokenizer(selected_model_path, checkpoint_path):
    model = AutoModelForCausalLM.from_pretrained(selected_model_path, device_map="auto")
    tokenizer = AutoTokenizer.from_pretrained(checkpoint_path, padding_side='left')

    model = PeftModel.from_pretrained(model, checkpoint_path) # comment for base model
    model = model.merge_and_unload()
    return model, tokenizer

In [ ]:
selected_model_path = "haoranxu/ALMA-7B-R"
checkpoint_path = "elena_data_v1/checkpoint-12"

#model_base, tokenizer_base = get_model_tokenizer_base(selected_model_path, checkpoint_path_base)
model, tokenizer = get_model_tokenizer(selected_model_path, checkpoint_path)

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Already found a `peft_config` attribute in the model. This will lead to having multiple adapters in the model. Make sure to know what you are doing!


## Inference

In [ ]:
bbb = ['industrial gauges that do not incorporate high activity sources']
pipe = pipeline(
task="text-generation",
model=model,
tokenizer=tokenizer,
max_length=1024,
do_sample = True,
return_full_text=False,
)
for i in range(len(bbb)):
    #src_lang = dev.iloc[i]['sources_lang']
    source = bbb[i]
    # if src_lang == 'RU':
    #     prompt = f"<s>[INST] Translate this from Russian to Chinese:\nRussian: {source} \nChinese: [/INST]"

    # else:

    prompt = f"<s>[INST] Translate this from English to Russian:\nEnglish: {source} \nRussian: [/INST]"
    translation = pipe(prompt, max_length=200)
    hyp = translation[0]['generated_text']
    print(i, "source = ", source)
    #print("references = ",  brackets.iloc[i].references)
    print("hyps = ",hyp)
    print("\n")

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


0 source =  industrial gauges that do not incorporate high activity sources
hyps =   промышленные датчики, не содержащие высокоактивные источники 




## Подсчет метрик

In [ ]:
from datasets import load_dataset

dataset = load_dataset("gsarti/wmt_vat", 'wmt16_en_ru')

In [ ]:
from datasets import load_dataset

dataset = load_dataset("Muennighoff/flores200", "eng_Latn-rus_Cyrl")

## 1 а) Инференс для базовой модели opus

In [ ]:
from transformers import MarianMTModel, MarianTokenizer

# Load the model and tokenizer for English to Russian translation
model_name = 'Helsinki-NLP/opus-mt-en-ru'
tokenizer = MarianTokenizer.from_pretrained(model_name)
model = MarianMTModel.from_pretrained(model_name)


In [ ]:
text = "Hello, how are you doing today?"  # Example text in English

# Tokenize the text
inputs = tokenizer(text, return_tensors="pt", max_length=512, truncation=True)

In [ ]:
# Generate the translation output
translated_tokens = model.generate(**inputs, max_length=512)

# Decode the translated tokens to get the Russian text
translated_text = tokenizer.decode(translated_tokens[0], skip_special_tokens=True)
print("Translated Text:", translated_text)

Translated Text: Привет, как дела?


In [ ]:
def inference_dataset(dataset, model, tokenizer):
    ref = []
    translated = []
    for i in range(500):
        # source = dataset['test'][i]['source']
        # reference = dataset['test'][i]['reference']
        source = dataset['test'][i]['en']
        reference = dataset['test'][i]['ru']
        inputs = tokenizer(source, return_tensors="pt", max_length=512, truncation=True)
        translated_tokens = model.generate(**inputs, max_length=512)
        translated_text = tokenizer.decode(translated_tokens[0], skip_special_tokens=True)

        print(i, "\n")
        print(source, "\n")
        print(reference, "\n")
        print(translated_text, "\n")
        ref.append(reference)
        translated.append(translated_text)
    inference_ = {
    'reference': ref,
    'hypothesis': translated
    }
    inference = pd.DataFrame(inference_)
    inference['source'] = list(dataset['test']['en'])[:500]
    return inference

In [ ]:
inference = inference_dataset(books, model, tokenizer)

0 

Industrial gauges that do not incorporate high activity sources 

Промышленные измерительные приборы, в которых не используются высокоактивные источники 

Промышленные датчики, не включающие высокоактивные источники 

1 

Some calibration sources, especially those of higher activity, are in specifically designed, shielded and collimated devices within large shielded facilities.  

Одни калибровочные источники, особенно с более высокой активностью, находятся в специально сконструированных экранированных и коллимированных устройствах, являющихся частью крупных экранированных установок.  

Некоторые калибровочные источники, особенно источники более высокой активности, находятся в специально спроектированных, защищенных и коллимированных устройствах на крупных защищенных установках. 

2 

Mendoza Valdezco, E. 

Mendoza Valdezco, E. 

Мендоса Вальдеско, Е. 

3 

Storage is defined as the holding of radioactive sources, spent fuel or radioactive waste in a facility that provides for thei

In [ ]:
inference.to_csv('elena_data_v1/inference/opus_test.csv')

In [ ]:
inference.to_csv('elena_data_v1/inference/opus_wmt16_flores.csv')

In [ ]:
inference = pd.read_csv('elena_data_v1/inference/opus_wmt16.csv')

In [ ]:
inference = pd.read_csv('elena_data_v1/inference/flores_alma.csv')

In [ ]:
inference_opus_test = pd.read_csv('elena_data_v1/inference/opus_test.csv')

In [ ]:
import sacrebleu
def calculate_sacrebleu(inference):
    chrf_scores = []
    for ref, hyp in zip(list(inference['reference']), inference['hypothesis']):
        score = sacrebleu.corpus_chrf([ref], [hyp])
        chrf_scores.append(score)

    # Print or process the chrF scores
    for i, score in enumerate(chrf_scores):
        print(f"Sentence {i+1}: chrF Score = {score}")
    return chrf_scores


In [ ]:
chrf_scores = calculate_sacrebleu(inference)

ref Промышленные измерительные приборы, в которых не используются высокоактивные источники
hyp Промышленные датчики, не включающие высокоактивные источники
ref Одни калибровочные источники, особенно с более высокой активностью, находятся в специально сконструированных экранированных и коллимированных устройствах, являющихся частью крупных экранированных установок. 
hyp Некоторые калибровочные источники, особенно источники более высокой активности, находятся в специально спроектированных, защищенных и коллимированных устройствах на крупных защищенных установках.
ref Mendoza Valdezco, E.
hyp Мендоса Вальдеско, Е.
ref Хранение определяют как содержание радиоактивных источников, отработавшего топлива или радиоактивных отходов на объекте, который обеспечивает их изоляцию, с целью их последующего извлечения [1]. 
hyp Хранение определяется как хранение радиоактивных источников, отработавшего топлива или радиоактивных отходов на объекте, обеспечивающем их/ее удержание, с целью извлечения [1].


TypeError: CHRF: Each element of `hyps` should be a string.

In [ ]:
chrf = [float(str(el)[-5:].strip()) for el in chrf_scores]
inference['Chrf'] = chrf
inference['Chrf'].mean()

6.3108200000000005

In [ ]:
import comet
def calculate_comet(inference):
# Загрузка предобученной модели COMET
    model_path = "Unbabel/wmt20-comet-da"
    model = comet.load_from_checkpoint(comet.download_model(model_path))

    # Подготовка данных для модели COMET
    data = [
        {"src": src, "mt": hyp, "ref": ref}
        for src, ref, hyp in zip(list(inference['source']), list(inference['reference']), list(inference['hypothesis']))
    ]

    # Прогоняем данные через модель и получаем оценки
    comet_scores = model.predict(data)
    return comet_scores


In [ ]:
comet_scores = calculate_comet(inference_opus_test)

In [ ]:
scores_comet = comet_scores['scores']
inference['comet'] = scores_comet
inference['comet'].mean()

0.418426441591233

In [ ]:
import nltk
from nltk.translate.meteor_score import meteor_score

# Убедитесь, что необходимые данные загружены
nltk.download('wordnet')
nltk.download('omw-1.4')

In [ ]:
def calculate_meteor(inference):
    meteor_scores = []
    for ref, hyp in zip(inference['reference'], inference['hypothesis']):
        ref_tokens = ref.split()  # Токенизируем эталонное предложение
        hyp_tokens = hyp.split()  # Токенизируем гипотетическое предложение
        score = meteor_score([ref_tokens], hyp_tokens)
        meteor_scores.append(score)

    # Print or process the METEOR scores
    for i, score in enumerate(meteor_scores):
        print(f"Sentence {i+1}: METEOR Score = {score:.4f}")
    return meteor_scores

In [ ]:
meteor_scores = calculate_meteor(inference)

Sentence 1: METEOR Score = 0.3628
Sentence 2: METEOR Score = 0.5016
Sentence 3: METEOR Score = 0.0000
Sentence 4: METEOR Score = 0.6486
Sentence 5: METEOR Score = 0.1163
Sentence 6: METEOR Score = 0.1667
Sentence 7: METEOR Score = 0.3377
Sentence 8: METEOR Score = 0.9375
Sentence 9: METEOR Score = 0.6726
Sentence 10: METEOR Score = 0.0000
Sentence 11: METEOR Score = 0.6052
Sentence 12: METEOR Score = 0.5783
Sentence 13: METEOR Score = 0.6138
Sentence 14: METEOR Score = 0.1607
Sentence 15: METEOR Score = 0.2015
Sentence 16: METEOR Score = 0.2439
Sentence 17: METEOR Score = 0.0000
Sentence 18: METEOR Score = 0.0000
Sentence 19: METEOR Score = 0.5094
Sentence 20: METEOR Score = 0.3799
Sentence 21: METEOR Score = 0.4297
Sentence 22: METEOR Score = 0.5663
Sentence 23: METEOR Score = 0.5000
Sentence 24: METEOR Score = 0.2619
Sentence 25: METEOR Score = 0.4993
Sentence 26: METEOR Score = 0.3744
Sentence 27: METEOR Score = 0.1493
Sentence 28: METEOR Score = 0.1562
Sentence 29: METEOR Score = 0

In [ ]:
inference['meteor'] = meteor_scores
inference.sort_values(by = ['meteor'])
inference['meteor'].mean()

0.3793974118168367

название модели  Sacreubleu  Comet  meteor

OPUS:           4.63           0.6        0.475

In [ ]:
Датасет FLORES  Sacreubleu   Comet      Meteor

OPUS            3.468       0.567       0.449

ALMA            3.01872     0.661       0.408

## Инференс fine-tune модели

In [ ]:
translated = []
pipe = pipeline(
task="text-generation",
model=model,
tokenizer=tokenizer,
max_length=1024,
do_sample = True,
return_full_text=False,
)
for i in range(500):
    source = books['test'][i]['en']
    reference = books['test'][i]['ru']
    #src_lang = dev.iloc[i]['sources_lang']
    # if src_lang == 'RU':
    #     prompt = f"<s>[INST] Translate this from Russian to Chinese:\nRussian: {source} \nChinese: [/INST]"

    # else:
    prompt = f"<s>[INST] Translate this from English to Russian:\nEnglish: {source} \nRussian: [/INST]"
    translation = pipe(prompt, max_length=200)
    hyp = translation[0]['generated_text']
    translated.append(hyp)
    print(i, "source = ", source)
    print(i, "reference = ", reference)
    #print("references = ",  brackets.iloc[i].references)
    print("hyps = ",hyp)
    print("\n")

In [ ]:
translated = [x.strip() for x in translated]
inference_ = {
'hypothesis': translated
}
inference = pd.DataFrame(inference_)
inference['reference'] = list(books['test']['ru'])[:500]
inference['source'] = list(books['test']['en'])[:500]
inference.to_csv('elena_data_v1/inference/alma_test.csv')

In [ ]:
inference

,hypothesis,reference,source,comet,meteor
0,"Промышленные датчики, не использующие источник...","Промышленные измерительные приборы, в которых ...",Industrial gauges that do not incorporate high...,0.039603,0.170455
1,"Некоторые калибровочные источники, особенно бо...","Одни калибровочные источники, особенно с более...","Some calibration sources, especially those of ...",0.403167,0.336050
2,"Мендоса Валдеско, Э.","Mendoza Valdezco, E.","Mendoza Valdezco, E.",-0.360742,0.000000
3,Хранение определяется как нахождение радиоакти...,Хранение определяют как содержание радиоактивн...,Storage is defined as the holding of radioacti...,0.663306,0.459959
4,Доступность и возможность перемещения источник...,"Пункт хранения должен быть легкодоступным, в т...","Easy access, including transfer of sources to ...",0.305115,0.114679
...,...,...,...,...,...
495,Тип излучения,Вид излучения\t,Radiation type \t,1.304485,0.250000
496,Дистрибьютор,Дистрибьютор\t,Distributor\t,1.327497,0.500000
497,Управление документами,Ведение учета\t,Records management \t,-0.588494,0.000000
498,У пользователя может возникнуть потребность в ...,"Пользователь может прийти к заключению, что од...",A user may find that one of these options will...,0.781109,0.178952


In [ ]:
def calculate_sacrebleu(inference):
    chrf_scores = []
    for ref, hyp in zip(list(inference['reference']), inference['hypothesis']):
        print("ref",  ref)
        print("hyp",  hyp)
        if hyp == '':
            chrf_scores.append(score)
        else:
            score = sacrebleu.corpus_chrf([ref], [hyp])
            chrf_scores.append(score)

    # Print or process the chrF scores
    for i, score in enumerate(chrf_scores):
        print(f"Sentence {i+1}: chrF Score = {score}")
    return chrf_scores

In [ ]:
sacrebleu_score = calculate_sacrebleu(inference)
sacr = [float(str(el)[-5:].strip()) for el in sacrebleu_score]
inference['sacr'] = sacr
inference['sacr'].mean()

ref Промышленные измерительные приборы, в которых не используются высокоактивные источники
hyp Промышленные датчики, не использующие источники высокой активности,
ref Одни калибровочные источники, особенно с более высокой активностью, находятся в специально сконструированных экранированных и коллимированных устройствах, являющихся частью крупных экранированных установок. 
hyp Некоторые калибровочные источники, особенно более активные, размещены в специально спроектированных, защищенных и направленных приборах, которые находятся в больших защищенных помещениях.
ref Mendoza Valdezco, E.
hyp Мендоса Валдеско, Э.
ref Хранение определяют как содержание радиоактивных источников, отработавшего топлива или радиоактивных отходов на объекте, который обеспечивает их изоляцию, с целью их последующего извлечения [1]. 
hyp Хранение определяется как нахождение радиоактивных источников, отработанного ядерного топлива или радиоактивных отходов в специально оборудованном помещении, где они находятся в з

5.293279999999999

In [ ]:
comet_score = calculate_comet(inference)
scores_comet = comet_score['scores']
inference['comet'] = scores_comet
inference['comet'].mean()

In [ ]:
scores_meteor = calculate_meteor(inference)
inference['meteor'] = scores_meteor
inference.sort_values(by = ['meteor'])
inference['meteor'].mean()

Sentence 1: METEOR Score = 0.1705
Sentence 2: METEOR Score = 0.3360
Sentence 3: METEOR Score = 0.0000
Sentence 4: METEOR Score = 0.4600
Sentence 5: METEOR Score = 0.1147
Sentence 6: METEOR Score = 0.1786
Sentence 7: METEOR Score = 0.4720
Sentence 8: METEOR Score = 0.0000
Sentence 9: METEOR Score = 0.3628
Sentence 10: METEOR Score = 0.0000
Sentence 11: METEOR Score = 0.3681
Sentence 12: METEOR Score = 0.2827
Sentence 13: METEOR Score = 0.4953
Sentence 14: METEOR Score = 0.0321
Sentence 15: METEOR Score = 0.1563
Sentence 16: METEOR Score = 0.1852
Sentence 17: METEOR Score = 0.1000
Sentence 18: METEOR Score = 0.0000
Sentence 19: METEOR Score = 0.3932
Sentence 20: METEOR Score = 0.2311
Sentence 21: METEOR Score = 0.2017
Sentence 22: METEOR Score = 0.5409
Sentence 23: METEOR Score = 0.5000
Sentence 24: METEOR Score = 0.3701
Sentence 25: METEOR Score = 0.4886
Sentence 26: METEOR Score = 0.0382
Sentence 27: METEOR Score = 0.1493
Sentence 28: METEOR Score = 0.6048
Sentence 29: METEOR Score = 0

0.3019795889527179

In [ ]:
inference['hyps_opus'] = inference_opus_test['hypothesis']

In [ ]:
_2_models = inference[['hypothesis', 'hyps_opus', 'comet', 'reference']].sample(n=20)

In [ ]:
for i in range(20):
    print(i)
    print("fine-tune", _2_models.iloc[i]['hypothesis'])
    print("opus", _2_models.iloc[i]['hyps_opus'])
    print("reference", _2_models.iloc[i]['reference'])
    print(_2_models.iloc[i]['comet'])

0
fine-tune Идеально, это должно быть страна производства, но если есть практическое решение, то это не обязательно.
opus В идеале это должна быть страна изготовления, но при наличии практической альтернативы в этом случае нет необходимости.


KeyError: 'reference'

0
fine-tune Следует отметить, что несоблюдение требований при хранении ДСРС на месте без применения необходимых мер предосторожности является наиболее распространенной причиной аварий и потери контроля над источниками.
opus Следует отметить, что наиболее частой причиной аварий и потери контроля за источниками является хранение ДСРС на месте без соблюдения соответствующих требований.
reference Следует отметить, что хранение ИЗРИ на местах без соблюдения соответствующих требований — наиболее распространенная причина аварий и утраты контроля над источниками. 
0.7505028247833252
1
fine-tune 
opus Я не знаю, что делать.
reference РИС. 4. Источники на основе жидкого 226Ra в стеклянных флаконах.
-2.0795860290527344
2
fine-tune Поставщики.
opus Поставщики.
reference — Поставщики. 
1.2850375175476074
3
fine-tune Каждый контейнер для отходов, в котором есть ДСР и он предназначен для хранения, должен соответствовать базовому набору требований (обычно называемым критериями приема отходов или требо

KeyboardInterrupt: 

0
fine-tune Промышленные датчики, не использующие источники высокой активности,
opus Промышленные датчики, не включающие высокоактивные источники
reference Промышленные измерительные приборы, в которых не используются высокоактивные источники
0.03960343450307846
1
fine-tune Калибровочные источники ионизирующего излучения, как правило, имеют меньшую активность, чем другие самозащищенные источники ионизирующего излучения, но некоторые из них содержат до 80 ТБк (2200 Кл) 137Cs.
opus Эти калибровочные облучатели в среднем являются источниками меньшей активности, чем другие самозащитные облучатели, однако некоторые из них содержат около 80 ТБк (2200 Сi) из 137С.
reference Эти калибровочные облучатели в среднем являются менее активными источниками, чем другие самоэкранированные облучатели, но некоторые из них содержат 137Cs с активностью около 80 ТБк (2200 Ки).
0.355345219373703
2
fine-tune Утечки или физически поврежденные источники (например, изогнутые, ржавые, треснувшие или сильно поцара

название модели  Sacreubleu  Comet  meteor

OPUS:           4.63           0.6        0.475
ALMA_fine_tuned:           4.11           0.653      0.419

In [ ]:
# sacrebleu - 4.11096
# comet - 0.6529
# meteor - 0.4193

## Инференс для базовой модели alma

In [ ]:
selected_model_path = "haoranxu/ALMA-7B-R"
model = AutoModelForCausalLM.from_pretrained(selected_model_path, device_map="auto")
tokenizer = AutoTokenizer.from_pretrained(selected_model_path, padding_side='left')



In [ ]:
translated = []
pipe = pipeline(
task="text-generation",
model=model,
tokenizer=tokenizer,
max_length=1024,
do_sample = True,
return_full_text=False,
)
for i in range(500):
    source = dataset['dev'][i]['sentence_eng_Latn']
    reference = dataset['dev'][i]['sentence_rus_Cyrl']
    #src_lang = dev.iloc[i]['sources_lang']
    # if src_lang == 'RU':
    #     prompt = f"<s>[INST] Translate this from Russian to Chinese:\nRussian: {source} \nChinese: [/INST]"

    # else:

    prompt = f"<s>[INST] Translate this from English to Russian:\nEnglish: {source} \nRussian: [/INST]"
    print(prompt)
    translation = pipe(prompt, max_length=500)
    hyp = translation[0]['generated_text']
    translated.append(hyp)
    print(i, "source = ", source)
    print(i, "reference = ", reference)
    #print("references = ",  brackets.iloc[i].references)
    print("hyps = ",hyp)
    print("\n")

In [ ]:
translated = [x.strip() for x in translated]
inference_ = {
'hypothesis': translated
}
inference = pd.DataFrame(inference_)
inference['reference'] = list(dataset['test']['reference'])[:500]
inference['source'] = list(dataset['test']['source'])[:500]
inference.to_csv('elena_data_v1/inference/opus_wmt16_alma_base.csv')

In [ ]:
sacrebleu_score = calculate_sacrebleu(inference)
sacr = [float(str(el)[-5:].strip()) for el in sacrebleu_score]
inference['sacr'] = sacr
inference['sacr'].mean()

Sentence 1: chrF Score = chrF2 = 6.25
Sentence 2: chrF Score = chrF2 = 4.24
Sentence 3: chrF Score = chrF2 = 0.00
Sentence 4: chrF Score = chrF2 = 10.64
Sentence 5: chrF Score = chrF2 = 6.10
Sentence 6: chrF Score = chrF2 = 0.00
Sentence 7: chrF Score = chrF2 = 0.00
Sentence 8: chrF Score = chrF2 = 0.00
Sentence 9: chrF Score = chrF2 = 0.00
Sentence 10: chrF Score = chrF2 = 3.97
Sentence 11: chrF Score = chrF2 = 0.00
Sentence 12: chrF Score = chrF2 = 8.33
Sentence 13: chrF Score = chrF2 = 7.58
Sentence 14: chrF Score = chrF2 = 0.00
Sentence 15: chrF Score = chrF2 = 8.77
Sentence 16: chrF Score = chrF2 = 3.47
Sentence 17: chrF Score = chrF2 = 8.47
Sentence 18: chrF Score = chrF2 = 0.00
Sentence 19: chrF Score = chrF2 = 4.27
Sentence 20: chrF Score = chrF2 = 9.80
Sentence 21: chrF Score = chrF2 = 0.00
Sentence 22: chrF Score = chrF2 = 2.40
Sentence 23: chrF Score = chrF2 = 7.69
Sentence 24: chrF Score = chrF2 = 5.56
Sentence 25: chrF Score = chrF2 = 7.35
Sentence 26: chrF Score = chrF2 =

4.21406

In [ ]:
comet_score = calculate_comet(inference)
scores_comet = comet_score['scores']
inference['comet'] = scores_comet
inference['comet'].mean()

In [ ]:
scores_meteor = calculate_meteor(inference)
inference['meteor'] = scores_meteor
inference.sort_values(by = ['meteor'])
inference['meteor'].mean()

Sentence 1: METEOR Score = 0.3989
Sentence 2: METEOR Score = 0.3600
Sentence 3: METEOR Score = 0.3289
Sentence 4: METEOR Score = 0.2000
Sentence 5: METEOR Score = 0.7146
Sentence 6: METEOR Score = 0.3377
Sentence 7: METEOR Score = 0.0495
Sentence 8: METEOR Score = 0.2960
Sentence 9: METEOR Score = 0.1402
Sentence 10: METEOR Score = 0.4869
Sentence 11: METEOR Score = 0.5034
Sentence 12: METEOR Score = 0.0500
Sentence 13: METEOR Score = 0.5901
Sentence 14: METEOR Score = 0.4009
Sentence 15: METEOR Score = 0.6320
Sentence 16: METEOR Score = 0.3602
Sentence 17: METEOR Score = 0.7472
Sentence 18: METEOR Score = 0.3497
Sentence 19: METEOR Score = 0.5292
Sentence 20: METEOR Score = 0.6726
Sentence 21: METEOR Score = 0.2107
Sentence 22: METEOR Score = 0.3566
Sentence 23: METEOR Score = 0.3505
Sentence 24: METEOR Score = 0.5978
Sentence 25: METEOR Score = 0.2950
Sentence 26: METEOR Score = 0.4291
Sentence 27: METEOR Score = 0.8377
Sentence 28: METEOR Score = 0.3840
Sentence 29: METEOR Score = 0

0.4210510968621195

In [ ]:
# название модели  Sacreubleu  Comet  meteor

#WMT-16
# OPUS:           4.63           0.6        0.475
# ALMA_fine_tuned 4.11           0.653      0.419
# ALMA_base:      4.21           0.638      0.421

#FLORAS(200)
# OPUS            3.468       0.567       0.449
# ALMA_base
# ALmA_fine_tuned 3.01872     0.661       0.408

#dev
#OPUS              6.31       0.418        0.379
#ALMA_base
#ALma_fine_tuned   5.293      0.334        0.302